In [35]:
%matplotlib inline
import torch



# ``torch.autograd``介绍

``torch.autograd``是PyTorch的自动微分引擎，为神经网络训练提供支持。
关于前向与bp，可以参看视频：[反向传播演算|附录深入学习第3章](https://www.youtube.com/watch?v=tIeHLnjs5U8)



# PyTorch中的使用

从``torchvision``中load模型resnet18（预训练），创建了一个随机的data与labes

以下是个简单的训练过程


In [3]:
import torch, torchvision
model = torchvision.models.resnet18(pretrained=True)
data = torch.rand(1, 3, 64, 64)
labels = torch.rand(1, 1000)

In [2]:
# forward pass
prediction = model(data) 

计算loss，并反向传播，此时<font color="red">Autograd将为每个模型参数计算梯度并将其存储在参数的``.grad``属性中</font>




In [ ]:
loss = (prediction - labels).sum()
loss.backward() # backward pass

In [5]:
# 设置优化算法
optim = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)

调用``.step()``进行梯度下降计算。optimizer通过存储在``.grad``中的梯度来调整每个参数。


In [6]:
optim.step() #gradient descent

# Differentiation in Autograd

设置``requires_grad=True``来启用``autograd``收集梯度。

下例为初始化数据，并设置函数：
\begin{align}Q = 3a^3 - b^2\end{align}

假设a，b为神经网络参数，Q为error，需要求Q对a，b的梯度：
\begin{align}\frac{\partial Q}{\partial a} = 9a^2\end{align}

\begin{align}\frac{\partial Q}{\partial b} = -2b\end{align}

方法1：显示传递参数`gradient`,此时我们求的是a，b的梯度，是个向量，与<font color=red>Q的shape相同</font>。

方法2：``Q.sum().backward()``：因为梯度只能为标量，所以要sum

In [36]:
# 方法2：
a = torch.tensor([2., 3.], requires_grad=True)
b = torch.tensor([6., 4.], requires_grad=True)
Q = 3*a**3 - b**2
Q.sum().backward()
print(9*a**2 == a.grad)
print(-2*b == b.grad)

tensor([True, True])
tensor([True, True])


In [37]:
# 方法1：
a = torch.tensor([2., 3.], requires_grad=True)
b = torch.tensor([6., 4.], requires_grad=True)
Q = 3*a**3 - b**2
external_grad = torch.tensor([1., 1.])
Q.backward(gradient=external_grad)

print(9*a**2 == a.grad)
print(-2*b == b.grad)

tensor([True, True])
tensor([True, True])


这个例子讲的非常清楚了：[详解Pytorch 自动微分里的（vector-Jacobian product）](https://zhuanlan.zhihu.com/p/65609544)

# 用autograd进行矢量计算
针对**<font color=red>向量函数</font>**$\vec{y}=f(\vec{x})$，求其Jacobian矩阵$J$:

\begin{align}J
     =
      \left(\begin{array}{cc}
      \frac{\partial \bf{y}}{\partial x_{1}} &
      ... &
      \frac{\partial \bf{y}}{\partial x_{n}}
      \end{array}\right)
     =
     \left(\begin{array}{ccc}
      \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{1}}{\partial x_{n}}\\
      \vdots & \ddots & \vdots\\
      \frac{\partial y_{m}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
      \end{array}\right)
\end{align}

`torch.autograd`用于计算vector-Jacobian product的引擎。意思是，给定任何向量$\vec{v}$，计算：$J^{T}\cdot \vec{v}$。

如果$v$恰好是一个scalar function（标量函数）的gradient：
\begin{align}
    l=g\left(\vec{y}\right) = \left(\begin{array}{ccc}\frac{\partial l}{\partial y_{1}} & \cdots & \frac{\partial l}{\partial y_{m}}\end{array}\right)^{T}
\end{align}

基于chain rule，这个vector-Jacobian product是$l$关于$\vec{x}$的梯度:
\begin{align}J^{T}\cdot \vec{v}=\left(\begin{array}{ccc}
      \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{1}}\\
      \vdots & \ddots & \vdots\\
      \frac{\partial y_{1}}{\partial x_{n}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
      \end{array}\right)\left(\begin{array}{c}
      \frac{\partial l}{\partial y_{1}}\\
      \vdots\\
      \frac{\partial l}{\partial y_{m}}
      \end{array}\right)=\left(\begin{array}{c}
      \frac{\partial l}{\partial x_{1}}\\
      \vdots\\
      \frac{\partial l}{\partial x_{n}}
      \end{array}\right)
\end{align}

**<font color=red>``external_grad`` represents $\vec{v}$</font>**

num_flat_features# Computational Graph
概念上讲，`autograd`记录了由[Function](https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function)对象组成的有向无换图(DAG) 中的data（tensor）与所有可执行操作（以及由此产生的新tensors）。**在此DAG中，叶子是输入张量，根是输出张量。通过从根到叶跟踪该图，可以使用链规则自动计算梯度**

前向传播中，autograd做2件事：

- 完成前向过程的计算并得到结果 
- 维护DAG中的``梯度函数``

在BP过程(调用`.backward()`)，autograde做以下事情：
- 从每一个`.grad_fn`计算梯度
- 累加在各自的相关张量`.grad`中
- 基于chain rule，传播到叶子节点。



Tips:
> DAG在PyTorch中是动态的。 需要注意的重要一点是，该图是从头开始重新创建的。每次 `.backward()`调用后，autograd开始填充新图。这正是允许您在模型中使用控制流语句的原因。您可以根据需要在每次迭代中更改形状，大小和操作。


下图是上例的DAG表示。箭头表示的是前向传播方向，节点（框）表示的是每一个前向forward pass的operation的backward函数。blue节点代表叶子张量：a与b
![自动梯度图](./assets/dag_autograd.png)


# Exclusion from the DAG
``requires_grad``控制是否需要计算梯度。设置为`False`，则不需要

In [38]:
x = torch.rand(5, 5)
y = torch.rand(5, 5)
z = torch.rand((5, 5), requires_grad=True)

a = x + y
print(f"Does `a` require gradients? : {a.requires_grad}")
b = x + z
print(f"Does `b` require gradients?: {b.requires_grad}")

Does `a` require gradients? : False
Does `b` require gradients?: True


## 冻结参数以微调预训练网络
利用`.requires_grad`冻结部分参数而更新其他参数，是目前一种比较常见的用法：对[预训练网络微调](https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html)

In [39]:
from torch import nn, optim

model = torchvision.models.resnet18(pretrained=True)

# Freeze all the parameters in the network
for param in model.parameters():
    param.requires_grad = False

In [40]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

resnet的classifier是最后一层`model.fc`，将其替换，训练我们设置的新的10个标签的数据。



In [ ]:
model.fc = nn.Linear(512, 10)

<font color=red>此处optimizer优化的是fc的参数</font>

In [ ]:
# Optimize only the classifier
optimizer = optim.SGD(model.fc.parameters(), lr=1e-2, momentum=0.9)

# Further readings:


-  [In-place operations & Multithreaded Autograd](https://pytorch.org/docs/stable/notes/autograd.html)
-  [Example implementation of reverse-mode autodiff](https://colab.research.google.com/drive/1VpeE6UvEPRz9HmsHh1KS0XxXjYu533EC)

